# Training Status Process
> Process to handle training data stream

In [ ]:
# | default_exp training_status_process

In [ ]:
# | export

import random
from datetime import datetime, timedelta
from os import environ
from time import sleep
from typing import *

import asyncio
from asyncer import asyncify
from fastapi import FastAPI
from fast_kafka_api.application import FastKafkaAPI
from sqlalchemy.exc import NoResultFound
from sqlmodel import Session, select, func

import airt_service
from airt_service.data.clickhouse import get_count
from airt_service.db.models import get_session_with_context, User, TrainingStreamStatus
from airt.logger import get_logger
from airt.patching import patch

[INFO] numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.


In [ ]:
import contextlib
import json
import threading
from pathlib import Path

import numpy as np
import pandas as pd
import pytest
import uvicorn
from confluent_kafka import Producer, Consumer
from _pytest.monkeypatch import MonkeyPatch

from airt_service.confluent import confluent_kafka_config, create_topics_for_user
from airt_service.db.models import create_user_for_testing
from airt_service.helpers import set_env_variable_context
from airt_service.server import create_ws_server
from airt_service.sanitizer import sanitized_print

23-01-12 13:11:23.636 [INFO] airt.executor.subcommand: Module loaded.


In [ ]:
test_username = create_user_for_testing()
display(test_username)

'doioivgoqd'

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
# | export


@patch(cls_method=True)
def _create(
    cls: TrainingStreamStatus,
    *,
    account_id: int,
    event: str,
    count: int,
    total: int,
    user: User,
    session: Session
) -> TrainingStreamStatus:
    """
    Method to create event

    Args:
        account_id: account id
        event: one of start, upload, end
        count: current count of rows in clickhouse db
        total: total no. of rows sent by user
        user: user object
        session: session object

    Returns:
        created object of type TrainingStreamStatus
    """
    training_event = TrainingStreamStatus(
        account_id=account_id, event=event, count=count, total=total, user=user
    )
    session.add(training_event)
    session.commit()
    return training_event

In [ ]:
throwaway_username = create_user_for_testing()
with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == throwaway_username)).one()

    test_end_event = TrainingStreamStatus._create(
        account_id=789, event="end", count=0, total=1000, user=user, session=session
    )
    session.refresh(test_end_event)
    display(test_end_event)
    assert test_end_event.id
    assert test_end_event.event == "end"

TrainingStreamStatus(uuid=UUID('d43be31c-b0da-4762-b297-ed4c73f35502'), id=20, count=0, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=12, event=<TrainingEvent.end: 'end'>, account_id=789, total=1000)

In [ ]:
# | export


def get_recent_event_for_user(
    username: str, session: Session
) -> List[TrainingStreamStatus]:
    """
    Get recent event for user

    Args:
        username: username of user to get recent events
        session: session object

    Returns:
        A list of recent events for given user
    """
    user = session.exec(select(User).where(User.username == username)).one()
    try:
        unique_account_ids = session.exec(
            select(TrainingStreamStatus.account_id)
            .where(TrainingStreamStatus.user == user)
            .distinct()
        )
    except NoResultFound:
        return []

    events = []
    for unique_account_id in unique_account_ids:
        try:
            events.append(
                session.exec(
                    select(TrainingStreamStatus)
                    .where(
                        TrainingStreamStatus.user == user,
                        TrainingStreamStatus.account_id == unique_account_id,
                    )
                    .order_by(TrainingStreamStatus.created.desc())  # type: ignore
                    .order_by(TrainingStreamStatus.id.desc())  # type: ignore
                    .limit(1)
                ).one()
            )
        except NoResultFound:
            pass
    return events

In [ ]:
end_count = 1_000_000

with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()
    actual = get_recent_event_for_user(username=test_username, session=session)
    assert actual == [], actual
    test_start_event = TrainingStreamStatus._create(
        account_id=999, event="start", count=0, total=10000, user=user, session=session
    )
    test_end_event = TrainingStreamStatus._create(
        account_id=999,
        event="end",
        count=10000,
        total=10000,
        user=user,
        session=session,
    )

    test_start_event = TrainingStreamStatus._create(
        account_id=666, event="start", count=0, total=1000, user=user, session=session
    )

    session.refresh(test_start_event)
    session.refresh(test_end_event)

    actual = get_recent_event_for_user(username=test_username, session=session)
    display(actual)
    assert len(actual) == 2
    assert actual[0] == test_end_event
    assert actual[1] == test_start_event

[TrainingStreamStatus(uuid=UUID('c2a4e23e-4712-4877-a496-17a448762014'), id=22, count=10000, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.end: 'end'>, account_id=999, total=10000),
 TrainingStreamStatus(uuid=UUID('6897d4ec-8410-4bf5-9748-b9a331137cbc'), id=23, count=0, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.start: 'start'>, account_id=666, total=1000)]

In [ ]:
# | export


def get_count_from_training_data_ch_table(account_id: int) -> int:
    """
    Get count of all rows for given account id from clickhouse table

    Args:
        account_id: account id to use

    Returns:
        Count for the given account id
    """
    return airt_service.data.clickhouse.get_count(
        account_id=account_id,
        username=environ["KAFKA_CH_USERNAME"],
        password=environ["KAFKA_CH_PASSWORD"],
        host=environ["KAFKA_CH_HOST"],
        port=int(environ["KAFKA_CH_PORT"]),
        database=environ["KAFKA_CH_DATABASE"],
        table=environ["KAFKA_CH_TABLE"],
        protocol=environ["KAFKA_CH_PROTOCOL"],
    )

In [ ]:
with MonkeyPatch.context() as monkeypatch:
    monkeypatch.setattr(
        "__main__.get_count_from_training_data_ch_table",
        lambda account_id: 999,
    )
    actual = get_count_from_training_data_ch_table(account_id=500)
    display(actual)
    assert actual == 999, actual

999

In [ ]:
# | export


async def process_recent_event(
    recent_event: TrainingStreamStatus,
    *,
    session: Session,
    end_timedelta: int = 30,
    fast_kafka_api_app: FastKafkaAPI
):
    """
    Process a single recent event for an username and an AccountId

    Args:
        recent_event: A recent event of type TrainingStreamStatus from database
        session: session object
        end_timedelta: timedelta in seconds to use to determine whether upload is over or not
    """
    prev_count = recent_event.count

    if recent_event.event == "end":
        # Check model training status started and start it if not already
        pass
    elif recent_event.event in ["start", "upload"]:
        curr_count = await asyncify(get_count_from_training_data_ch_table)(
            account_id=recent_event.account_id
        )
        curr_check_on = datetime.utcnow()

        #         logger.info(f"{recent_event=}")
        if (
            curr_count == prev_count
            and curr_check_on - recent_event.created > timedelta(seconds=end_timedelta)
        ):
            to_update_event = "end"
            # Start model training status
        elif curr_count != prev_count:
            to_update_event = "upload"
        else:
            return
        upload_event = await asyncify(TrainingStreamStatus._create)(  # type: ignore
            account_id=recent_event.account_id,
            event=to_update_event,
            count=curr_count,
            total=recent_event.total,
            user=recent_event.user,
            session=session,
        )
        # send status msg to kafka
        #         training_data_status = TrainingDataStatus(
        #             AccountId=recent_event.account_id,
        #             no_of_records=curr_count,
        #             total_no_of_records=recent_event.total,
        #         )
        await fast_kafka_api_app.to_infobip_training_data_status(
            account_id=recent_event.account_id,
            no_of_records=curr_count,
            total_no_of_records=recent_event.total,
        )

In [ ]:
dummy_fast_kafka_api = FastKafkaAPI(FastAPI())


async def dummy_to_infobip_training_data_status(*args, **kwargs):
    logger.info("from dummy func for to_infobip_training_data_status")


dummy_fast_kafka_api.to_infobip_training_data_status = (
    dummy_to_infobip_training_data_status
)


with get_session_with_context() as session:
    test_upload_event = TrainingStreamStatus._create(
        account_id=666,
        event="upload",
        count=1000,
        total=1000,
        user=user,
        session=session,
    )
    with MonkeyPatch.context() as monkeypatch:

        monkeypatch.setattr(
            "__main__.get_count_from_training_data_ch_table",
            lambda account_id: 1000,
        )

        user = session.exec(select(User).where(User.username == test_username)).one()
        test_recent_events = get_recent_event_for_user(
            username=test_username, session=session
        )
        display(test_recent_events)

        await process_recent_event(
            test_recent_events[0],
            session=session,
            fast_kafka_api_app=dummy_fast_kafka_api,
        )
        changed_recent_events = get_recent_event_for_user(
            username=test_username, session=session
        )
        assert test_recent_events == changed_recent_events

        sleep(12)
        await process_recent_event(
            test_recent_events[1],
            session=session,
            end_timedelta=10,
            fast_kafka_api_app=dummy_fast_kafka_api,
        )
        changed_recent_events = get_recent_event_for_user(
            username=test_username, session=session
        )
        display(changed_recent_events)
        assert test_recent_events[0] == changed_recent_events[0]
        assert changed_recent_events[1].account_id == 666
        assert changed_recent_events[1].event == "end"

[TrainingStreamStatus(uuid=UUID('c2a4e23e-4712-4877-a496-17a448762014'), id=22, count=10000, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.end: 'end'>, account_id=999, total=10000),
 TrainingStreamStatus(uuid=UUID('2aacc708-d463-4f5a-85a0-16cb8fc995a7'), id=24, count=1000, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.upload: 'upload'>, account_id=666, total=1000)]

23-01-12 13:11:36.492 [INFO] __main__: recent_event=TrainingStreamStatus(uuid=UUID('2aacc708-d463-4f5a-85a0-16cb8fc995a7'), id=24, count=1000, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.upload: 'upload'>, account_id=666, total=1000)
23-01-12 13:11:36.498 [INFO] __main__: from dummy func for to_infobip_training_data_status


[TrainingStreamStatus(uuid=UUID('c2a4e23e-4712-4877-a496-17a448762014'), id=22, count=10000, created=datetime.datetime(2023, 1, 12, 13, 11, 24), user_id=11, event=<TrainingEvent.end: 'end'>, account_id=999, total=10000),
 TrainingStreamStatus(uuid=UUID('ae9e69e8-4958-40c9-84ef-4ba7677ebbcb'), id=25, count=1000, created=datetime.datetime(2023, 1, 12, 13, 11, 36), user_id=11, event=<TrainingEvent.end: 'end'>, account_id=666, total=1000)]

In [ ]:
# | export


async def process_training_status(username: str, fast_kafka_api_app: FastKafkaAPI):
    """
    An infinite loop to keep track of training_data uploads from user

    Args:
        username: username of user to track training data uploads
    """
    while True:
#         logger.info(f"Starting the process loop")
        with get_session_with_context() as session:
            recent_events = await asyncify(get_recent_event_for_user)(username, session)
            for recent_event in recent_events:
                await process_recent_event(
                    recent_event, session=session, fast_kafka_api_app=fast_kafka_api_app
                )
        await asyncio.sleep(random.randint(1, 4))

In [ ]:
definitions = [
    "appLaunch",
    "sign_in",
    "sign_out",
    "add_to_cart",
    "purchase",
    "custom_event_1",
    "custom_event_2",
    "custom_event_3",
]


applications = ["DriverApp", "PUBG", "COD"]


def generate_n_rows_for_training_data(n: int, seed: int = 42):
    rng = np.random.default_rng(seed=seed)
    #     account_id = rng.choice([4000, 5000, 500], size=n)
    account_id = 6000
    definition_id = rng.choice(definitions, size=n)
    application = rng.choice(applications, size=n)
    occurred_time_ticks = rng.integers(
        datetime(year=2022, month=1, day=1).timestamp() * 1000,
        datetime(year=2022, month=11, day=1).timestamp() * 1000,
        size=n,
    )
    occurred_time = pd.to_datetime(occurred_time_ticks, unit="ms").strftime(
        "%Y-%m-%dT%H:%M:%S.%f"
    )
    person_id = rng.integers(n // 10, size=n)

    df = pd.DataFrame(
        {
            "AccountId": account_id,
            "Application": application,
            "DefinitionId": definition_id,
            "OccurredTimeTicks": occurred_time_ticks,
            "OccurredTime": occurred_time,
            "PersonId": person_id,
        }
    )
    return json.loads(df.to_json(orient="records"))


generate_n_rows_for_training_data(100)[-1]

{'AccountId': 6000,
 'Application': 'COD',
 'DefinitionId': 'sign_in',
 'OccurredTimeTicks': 1649146037462,
 'OccurredTime': '2022-04-05T08:07:17.462000',
 'PersonId': 4}

In [ ]:
class Server(uvicorn.Server):
    def install_signal_handlers(self):
        pass

    @contextlib.contextmanager
    def run_in_thread(self):
        thread = threading.Thread(target=self.run)
        thread.start()
        try:
            while not self.started:
                sleep(1e-3)
            yield
        finally:
            self.should_exit = True
            thread.join()


def delivery_report(err, msg):
    """Called once for each message produced to indicate delivery result.
    Triggered by poll() or flush()."""
    if err is not None:
        sanitized_print("Message delivery failed: {}".format(err))
    else:
        #         sanitized_print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))
        pass

In [ ]:
create_topics_for_user(username=test_username)


def test_process_training_status():
    logger.info("I am done at tests")
    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == test_username)).one()
        test_start_event = TrainingStreamStatus._create(
            account_id=6000,
            event="start",
            count=0,
            total=1000,
            user=user,
            session=session,
        )
        session.add(test_start_event)
        session.commit()

        p = Producer(confluent_kafka_config)
        msg_count = 1000
        training_data = generate_n_rows_for_training_data(msg_count, seed=999)
        for i in range(msg_count):
            p.produce(
                f"{test_username}_training_data",
                json.dumps(training_data[i]).encode("utf-8"),
                on_delivery=delivery_report,
            )
        p.flush()

    while True:
        sleep(5)
        with get_session_with_context() as session:
            user = session.exec(
                select(User).where(User.username == test_username)
            ).one()
            event = session.exec(
                select(TrainingStreamStatus)
                .where(TrainingStreamStatus.user == user)
                .where(TrainingStreamStatus.account_id == 6000)
                .order_by(TrainingStreamStatus.id.desc())
                .limit(1)
            ).one()
            logger.info(f"event in test is {event}")
            if event.event == "end":
                display(f"All events for account id {6000}")
                all_events = session.exec(
                    select(TrainingStreamStatus)
                    .where(TrainingStreamStatus.user == user)
                    .where(TrainingStreamStatus.account_id == 6000)
                )
                display([e for e in all_events])
                break


with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
    with MonkeyPatch.context() as monkeypatch:
        monkeypatch.setattr(
            "__main__.get_count_from_training_data_ch_table",
            lambda account_id: 999,
        )
        app, fast_kafka_api_app = create_ws_server(
            assets_path=Path("../assets"), start_process_for_username=None
        )

        @fast_kafka_api_app.run_in_background()
        async def startup_event():
            await process_training_status(
                username=test_username, fast_kafka_api_app=fast_kafka_api_app
            )

        config = uvicorn.Config(app, host="127.0.0.1", port=6006, log_level="debug")
        server = Server(config=config)

        with server.run_in_thread():
            # Server started.
            sanitized_print("server started")
            test_process_training_status()

        sanitized_print("server stopped")
        # Server stopped.

23-01-12 13:11:36.684 [INFO] airt_service.confluent: Topic doioivgoqd_start_training_data created
23-01-12 13:11:36.684 [INFO] airt_service.confluent: Topic doioivgoqd_training_data created
23-01-12 13:11:36.685 [INFO] airt_service.confluent: Topic doioivgoqd_realitime_data created
23-01-12 13:11:36.686 [INFO] airt_service.confluent: Topic doioivgoqd_training_data_status created
23-01-12 13:11:36.686 [INFO] airt_service.confluent: Topic doioivgoqd_training_model_status created
23-01-12 13:11:36.687 [INFO] airt_service.confluent: Topic doioivgoqd_model_metrics created
23-01-12 13:11:36.687 [INFO] airt_service.confluent: Topic doioivgoqd_prediction created


%4|1673529096.540|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673529096.540|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


23-01-12 13:11:36.968 [INFO] airt_service.server: kafka_config={'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092', 'group_id': 'kumaran-airt-service-kafka-1:9092_group', 'auto_offset_reset': 'earliest'}


INFO:     Started server process [4120]
INFO:     Waiting for application startup.


23-01-12 13:11:37.074 [INFO] fast_kafka_api._components.asyncapi: Keeping the old async specifications at: 'asyncapi/spec/asyncapi.yml'
23-01-12 13:11:37.075 [INFO] fast_kafka_api._components.asyncapi: Skipping generating async documentation in '/work/airt-service/notebooks/asyncapi/docs'
23-01-12 13:11:37.076 [INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
23-01-12 13:11:37.086 [INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
23-01-12 13:11:37.096 [INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
23-01-12 13:11:37.105 [INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092'}'
23-01-12 13:11

...

[TrainingStreamStatus(uuid=UUID('bf703b1a-7471-4bb4-b2ec-55ccf2d8aca1'), id=26, count=0, created=datetime.datetime(2023, 1, 12, 13, 11, 37), user_id=11, event=<TrainingEvent.start: 'start'>, account_id=6000, total=1000),
 TrainingStreamStatus(uuid=UUID('16133704-12c5-4ca7-b299-e83a5ca3fa12'), id=27, count=999, created=datetime.datetime(2023, 1, 12, 13, 11, 39), user_id=11, event=<TrainingEvent.upload: 'upload'>, account_id=6000, total=1000),
 TrainingStreamStatus(uuid=UUID('d8f70011-9027-41ef-9fe5-a87ec8c06116'), id=28, count=999, created=datetime.datetime(2023, 1, 12, 13, 12, 12), user_id=11, event=<TrainingEvent.end: 'end'>, account_id=6000, total=1000)]

INFO:     Shutting down
INFO:     Waiting for application shutdown.


23-01-12 13:12:13.630 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
23-01-12 13:12:13.631 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
23-01-12 13:12:13.632 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
23-01-12 13:12:13.633 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-01-12 13:12:13.633 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
23-01-12 13:12:13.635 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-01-12 13:12:13.636 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
23-01-12 13:12:13.637 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-01-12 13:12:13.637 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished

INFO:     Application shutdown complete.
INFO:     Finished server process [4120]


server stopped
